Очистка данных в соответствии с выводами полученными после EDA.

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

# загружаем данные

df = pd.read_excel("data.xlsx", engine='openpyxl')

# удаляем неинформативные столбцы (если есть)
if 'Unnamed: 0' in df.columns:
    df.drop(columns=['Unnamed: 0'], inplace=True)

# выделение целевых переменных
target_columns = ['IC50, mM', 'CC50, mM', 'SI']
X = df.drop(columns=target_columns)
y = df[target_columns]

# оставляем только числовые признаки
numeric_cols = X.select_dtypes(include=np.number).columns
X = X[numeric_cols]

# проверяем на пропуски
print(" Проверка на пропущенные значения.")
missing = X.isnull().sum()
if missing.any():
    print("Предупреждение: найдены пропущенные значения!")
    print(missing[missing > 0])
else:
    print("Пропущенных значений нет.")

# заполнение пропусков медианой
print("Обработка пропущенных значений.")
imputer = SimpleImputer(strategy='median')
X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

# объединяем обратно для обработки
df_combined = pd.concat([X_imputed, y], axis=1)

# функция замены выбросов по IQR
def cap_outliers_iqr(df, columns):
    df_capped = df.copy()
    for col in columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df_capped[col] = df_capped[col].clip(lower=lower_bound, upper=upper_bound)
    return df_capped

# применяем функцию к числовым колонкам
print("Обработка выбросов методом IQR.")
df_capped = cap_outliers_iqr(df_combined, df_combined.columns)

# функция удаления сильно коррелированных признаков
def remove_high_corr_features(df, threshold=0.8):
    corr_matrix = df.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
    print(f"Удаляем признаки с высокой корреляцией (> {threshold}): {to_drop}")
    return df.drop(columns=to_drop)

# удаляем коррелированные признаки (только среди X, без целевых)
X_clean = remove_high_corr_features(df_capped.drop(columns=target_columns))
df_clean = pd.concat([X_clean, df_capped[target_columns]], axis=1)

# нормализуем данные
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_clean)

# сохранеем данные полученные после обработки в CSV
output_path = "cleaned_data.csv"
df_clean.to_csv(output_path, index=False)

# вывод информации о результате
print("Информация о результирующем датасете:")
print(f"Количество строк: {df_clean.shape[0]}")
print(f"Количество столбцов: {df_clean.shape[1]}")
print("Первые 5 строк:")
print(df_clean.head())

 Проверка на пропущенные значения.
Предупреждение: найдены пропущенные значения!
MaxPartialCharge       3
MinPartialCharge       3
MaxAbsPartialCharge    3
MinAbsPartialCharge    3
BCUT2D_MWHI            3
BCUT2D_MWLOW           3
BCUT2D_CHGHI           3
BCUT2D_CHGLO           3
BCUT2D_LOGPHI          3
BCUT2D_LOGPLOW         3
BCUT2D_MRHI            3
BCUT2D_MRLOW           3
dtype: int64
Обработка пропущенных значений.
Обработка выбросов методом IQR.
Удаляем признаки с высокой корреляцией (> 0.8): ['MaxEStateIndex', 'HeavyAtomMolWt', 'ExactMolWt', 'NumValenceElectrons', 'MaxAbsPartialCharge', 'MinAbsPartialCharge', 'FpDensityMorgan2', 'FpDensityMorgan3', 'BCUT2D_CHGLO', 'BCUT2D_LOGPHI', 'BCUT2D_LOGPLOW', 'BCUT2D_MRHI', 'BertzCT', 'Chi0', 'Chi0n', 'Chi0v', 'Chi1', 'Chi1n', 'Chi1v', 'Chi2n', 'Chi2v', 'Chi3n', 'Chi3v', 'Chi4n', 'Chi4v', 'HallKierAlpha', 'Ipc', 'Kappa1', 'Kappa2', 'Kappa3', 'LabuteASA', 'SMR_VSA7', 'SlogP_VSA11', 'SlogP_VSA12', 'SlogP_VSA4', 'SlogP_VSA5', 'SlogP_VSA6', 